In [1]:
import sys
import os
import pymysql
import xmltodict
import redis
import requests
#from datetime import datetime
import datetime
from collections import namedtuple
import logging
from time import sleep
import time

In [6]:
import requests
import xmltodict

In [44]:
basic_url="http://plus.kipris.or.kr/openapi/rest/RegistrationService/"
taskId="registrationInfo"
reg_number='registrationNumber'
number='1016988410000'
api_key = "Y74vSqogy7fkw71F26g146N4s9Harc7sLqm4ONkWHWE="
api_key_2= "S5sUeUcewvE0=dLF=a9IKgK72zEAvD5bkiPciv4BIEU="
url =f'{basic_url}{taskId}?{reg_number}={number}&accessKey={api_key}'
url
url_2=f'{basic_url}{taskId}?{reg_number}={number}&accessKey={api_key_2}'
url_2

'http://plus.kipris.or.kr/openapi/rest/RegistrationService/registrationInfo?registrationNumber=1016988410000&accessKey=S5sUeUcewvE0=dLF=a9IKgK72zEAvD5bkiPciv4BIEU='

In [23]:
xd=xmltodict.parse(con)

In [47]:
rr=requests.get(url_2)

In [49]:
rrr=rr.content

In [50]:
rrrr=xmltodict.parse(rrr)

In [52]:
its=list(rrrr['response']['body'])[0]

In [53]:
its_xd=rrrr['response']['body'][f'{its}']

In [68]:
for key, item in its_xd['registrationInfo'].items():
    print(key)

registrationRightInfo
registrationRightHolderInfo
registrationRightRankInfo
registrationFeeInfo
registrationLastRightHolderInfo


In [80]:
body=list(xmltodict.parse(rrr)['response']['body'])[0]
content=xmltodict.parse(rrr)['response']['body'][f'{body}']
detail_name=list(content)[0]
content=content[detail_name]

In [87]:
right_idc_info=list(content['registrationRightInfo'].values())

In [90]:
right_info_=list(content['registrationRightInfo'])
right_info_

['registrationNumber',
 'regReferenceNumber',
 'registrationDate',
 'assessmentDate',
 'expirationDate',
 'terminationCauseName',
 'terminationDate',
 'applicationNumber',
 'appReferenceNumber',
 'applicationDate',
 'publicationNumber',
 'publicationDate',
 'internationRegistrationNumber',
 'internationRegistrationDate',
 'originalApplicationNumber',
 'originalApplicationDate',
 'classCode',
 'titleOfInvention',
 'titleOfInventionEng',
 'claimCount',
 'priorityCountry',
 'priorityDate',
 'priorityCount']

In [88]:
right_idc_info

['1016988410000',
 None,
 '20170117',
 '20161125',
 '20350519',
 None,
 None,
 '1020150069853',
 None,
 '20150519',
 None,
 '20170123',
 None,
 None,
 None,
 None,
 'D06F 58/26',
 '건조 장치',
 'Drying device',
 '11',
 None,
 None,
 '0']

In [91]:
right_idc_info=list(content['registrationRightInfo'].values())

In [96]:
right_idc_info

['1016988410000',
 None,
 '20170117',
 '20161125',
 '20350519',
 None,
 None,
 '1020150069853',
 None,
 '20150519',
 None,
 '20170123',
 None,
 None,
 None,
 None,
 'D06F 58/26',
 '건조 장치',
 'Drying device',
 '11',
 None,
 None,
 '0']

In [86]:
def post_db(self, query, data, conn_data,commit=True):
    tries=0
    while True:
        try:
            conn = pymysql.connect(host=conn_data['host'],
                                    port=conn_data['port'],
                                    user = conn_data['user'],
                                    password=conn_data['password'],
                                    database=conn_data['database'],
                                    connect_timeout=60)
            with conn.cursor() as curs:
                if (type(data[0]) is list) & (len(data) !=1):
                    curs.executemany(query,data)
                elif (type(data[0]) is list) & (len(data)==1):
                    curs.execute(query,data)
            if commit:
                conn.commit()
            conn.close()
            return True
        except:
            if tries ==12:
                conn.close()
                return False
            tries +=1
            sleep(5)
def add_info(orddict,number=None,only_none=False):
    if not only_none:
        orddict['number'] = number
        orddict['date'] = list(redis.smembers("work_date"))[0]
        
    for response in orddict.items():
        if response[1]==None:
            if (response[0] == 'trialNumberr') | (response[0] == 'registrationNumber'):
                orddict[f'{response[0]}'] = 'N/A'
            orddict[f'{response[0]}'] = ''
def post_many(flush=False,target=None,data=None):
    if not flush:
        if target not in _many.keys():
            if isinstance(data, list):
                if isinstance(data[0], list):
                    _many[target]=data
                else:
                    print('post_many processing error')
            else:
                print('post_many data type error')
        else:
            print('post_many target processing error')
        if len(_many[target]) ==100:
            post_db(match_info[target][0], _many[target],match_info[target][1])
            _many.pop(target)
            print('db insert complete, delete info...')
    
    if flush:
        for _key in sorted(_many.keys()):
            try:
                post_db(match_info[_key][0], _many[_key],match_info[_key][1])
            except:
                print(_key)
                print(_many[_key])
_many={}
conn_data = {
            'host' : '127.0.0.1',
            'port' : 3306,
            'user': 'root',
            'password':'123',
            'database': "KIPRIS"
        }
match_info = {
            'right_info': [rightholder_info, conn_data],
            'reg_fee': [registrationFee_query, conn_data],
            'reg_rank': [rightRank_query, conn_data],
            'reg_rightA': [rightHolderA_query, conn_data],
            'reg_rightB': [rightHolderB_query, conn_data]
        }
app_from_reg= """
        select applicationNumber
        from KIPO_ADMIN_BIBLIOGRAPHIC
        where registerNumber = %s
        """
rightHolderA_query = """
INSERT INTO REGISTRATION_RIGHT_HOLDER_A(
    RANK_NUMBER,
    RANK_CORRELATOR_SERIAL_NUMBER,
    RANK_CORRELATOR_TYPE,
    RANK_CORRELATOR_NAME,
    RANK_CORRELATOR_ADDRESS,
    REGISTRATION_NUMBER,
    TRANSFER_DATE
)
VALUES (%s, %s, %s, %s, %s,%s, %s)
ON DUPLICATE KEY UPDATE
    RANK_CORRELATOR_NAME = VALUES(RANK_CORRELATOR_NAME),
    RANK_CORRELATOR_ADDRESS= VALUES(RANK_CORELATOR_ADDRESS),
    TRANSFER_DATE=VALUES(TRANSFER_DATE),
    MOD_DT=SYSDATE()
"""
rightHolderB_query="""
INSERT INTO REGISTRATION_RIGHT_HOLDER_B (
    RANK_NUMBER,
    DOCUMENT_NAME,
    RECEIPT_DATE,
    REGISTRATION_CAUSE_NAME,
    INDECATION_OF_EVENT,
    REGISTRATION_NUMBER,
    TRANSFER_DATE            
)
VALUES (%s,%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
    DOCUMENT_NAME = VALUES(DOCUMENT_NAME),
    RECEIPT_DATE = VALUES(RECEIPT_DATE),
    REGISTRATION_CAUSE_NAME = VALUES(REGISTRATION_CAUSE_NAME),
    INDICATION_OF_EVENT = VALUES(INDICATION_OF_EVENT),
    TRANSFER_DATE = VALUES(TRANSFER_DATE),
    MOD_DT=SYSDATE()
"""

rightRank_query = """
INSERT INTO REGISTRATION_RIGHT_RANK (
    RANK_NUMBER
    , PERTINENT_PARTITION
    , DOCUMENT_NAME
    , ORIGINAL_REGISTRATION_NUMBER
    , REGISTRATION_PURPOSE
    , REGISTRATION_DATE
    , REGISTRATION_CAUSE_NAME
    , REGISTRATION_CAUSE_DATE
    , RECEIPT_NUMBER
    , RECEIPT_DATE
    , DISAPPEARANCE_FLAG
    , DISAPPEARANCE_CAUSE_NAME
    , DISAPPEARANCE_DATE
    , INTERNATIONAL_REG_RECORD_DATE_MD
    , EXPIRATION_DATE_MD
    , LATEST_RENEWAL_DATE_MD
    , SUB_DESIGNATION_DATE_MD
    , REGISTRATION_NUMBER
    , TRANSFER_DATE
    )
VALUES (%s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
DOCUMENT_NAME = VALUES(DOCUMENT_NAME)
, ORIGINAL_REGISTRATION_NUMBER = VALUES(ORIGINAL_REGISTRATION_NUMBER)
, REGISTRATION_PURPOSE = VALUES(REGISTRATION_PURPOSE)
, REGISTRATION_DATE = VALUES(REGISTRATION_DATE)
, REGISTRATION_CAUSE_NAME = VALUES(REGISTRATION_CAUSE_NAME)
, REGISTRATION_CAUSE_DATE = VALUES(REGISTRATION_CAUSE_DATE)
, RECEIPT_NUMBER = VALUES(RECEIPT_NUMBER)
, RECEIPT_DATE = VALUES(RECEIPT_DATE)
, DISAPPEARANCE_FLAG = VALUES(DISAPPEARANCE_FLAG)
, DISAPPEARANCE_CAUSE_NAME = VALUES(DISAPPEARANCE_CAUSE_NAME)
, DISAPPEARANCE_DATE = VALUES(DISAPPEARANCE_DATE)
, INTERNATIONAL_REG_RECORD_DATE_MD = VALUES(INTERNATIONAL_REG_RECORD_DATE_MD)
, EXPIRATION_DATE_MD = VALUES(EXPIRATION_DATE_MD)
, LATEST_RENEWAL_DATE_MD = VALUES(LATEST_RENEWAL_DATE_MD)
, SUB_DESIGNATION_DATE_MD = VALUES(SUB_DESIGNATION_DATE_MD)
, TRANSFER_DATE = VALUES(TRANSFER_DATE)
, MOD_DT = SYSDATE()
"""

rightholder_info = '''
INSERT INTO RIGHT_HOLDER (
    PERTINENT_PARTITION
    , RANK_NUMBER
    , REGISTRATION_DATE
    , RANK_CORRELATOR_SERIAL_NUMBER
    , RANK_CORRELATOR_NUMBER
    , RANK_CORRELATOR_TYPE
    , RANK_CORRELATOR_NAME
    , RANK_CORRELATOR_ADDRESS
    , TRANSFER_DATE
    , REGISTRATION_NUMBER
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
    REGISTRATION_NUMBER = VALUES(REGISTRATION_NUMBER)
    , PERTINENT_PARTITION = VALUES(PERTINENT_PARTITION)
    , RANK_CORRELATOR_TYPE = VALUES(RANK_CORRELATOR_TYPE)
    , RANK_CORRELATOR_SERIAL_NUMBER = VALUES(RANK_CORRELATOR_SERIAL_NUMBER)
    , RANK_NUMBER = VALUES(RANK_NUMBER)
    , RANK_CORRELATOR_NAME = VALUES(RANK_CORRELATOR_NAME)
    , RANK_CORRELATOR_ADDRESS = VALUES(RANK_CORRELATOR_ADDRESS)
    , RANK_CORRELATOR_NUMBER = VALUES(RANK_CORRELATOR_NUMBER)
    , REGISTRATION_DATE = VALUES(REGISTRATION_DATE)
    , TRANSFER_DATE = VALUES(TRANSFER_DATE)
    , MOD_DT = SYSDATE()
'''
registrationFee_query = '''
INSERT INTO REGISTRATION_FEE (
    REGISTRATION_DATE
    , START_ANNUAL
    , LAST_ANNUAL
    , PAYMENT_DEGREE
    , PAYMENT_FEE
    , PAYMENT_DATE
    , REGISTRATION_NUMBER
    , TRANSFER_DATE
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
    START_ANNUAL = VALUES(START_ANNUAL)
    , LAST_ANNUAL = VALUES(LAST_ANNUAL)
    , PAYMENT_DEGREE = VALUES(PAYMENT_DEGREE)
    , PAYMENT_FEE = VALUES(PAYMENT_FEE)
    , PAYMENT_DATE = VALUES(PAYMENT_DATE)
    , TRANSFER_DATE = VALUES(TRANSFER_DATE)
    , MOD_DT = SYSDATE()
        ''' 

NameError: name 'rightholder_info' is not defined

In [66]:
content['registrationRightHolderInfo']

{'registrationRightHolderInfoA': {'rankNumber': '1',
  'rankCorrelatorSerialNumber': '1',
  'rankCorrelatorType': '권리자',
  'rankCorrelatorName': '주식회사 코나솔',
  'rankCorrelatorAddress': '충청남도 당진...'},
 'registrationRightHolderInfoB': {'rankNumber': '1',
  'documentName': '설정등록',
  'receiptDate': '20170117',
  'registrationCauseName': None,
  'indicationOfEvent': None}}

In [70]:
type(content['registrationRightHolderInfo']['registrationRightHolderInfoA'])

dict

In [71]:
type(list(content['registrationRightHolderInfo']['registrationRightHolderInfoA']))

list

In [63]:
content.keys()

dict_keys(['registrationRightInfo', 'registrationRightHolderInfo', 'registrationRightRankInfo', 'registrationFeeInfo', 'registrationLastRightHolderInfo'])

In [39]:
cont=list(xd['response']['body'][f'{its}'])[0]

In [40]:
cont

'registrationInfo'

In [42]:
aa=its_xd[cont]

In [43]:
aa

{'registrationRightHolderInfo': None}

In [18]:
content=request_get['response']['body']
content['items']['registrationInfo']['registrationRightHolderInfo']

In [7]:
request_get=requests.get(url)
request_get=xmltodict.parse(request_get.content)
content=request_get['response']['body']
detail_name=list(content)[0]
detail_name

'items'

In [72]:
right_a=list(content['registrationRightHolderInfo']['registrationRightHolderInfoA'])
right_a

['rankNumber',
 'rankCorrelatorSerialNumber',
 'rankCorrelatorType',
 'rankCorrelatorName',
 'rankCorrelatorAddress']

In [73]:
right_info_=list(content['registrationRightInfo'])

In [74]:
right_info_

['registrationNumber',
 'regReferenceNumber',
 'registrationDate',
 'assessmentDate',
 'expirationDate',
 'terminationCauseName',
 'terminationDate',
 'applicationNumber',
 'appReferenceNumber',
 'applicationDate',
 'publicationNumber',
 'publicationDate',
 'internationRegistrationNumber',
 'internationRegistrationDate',
 'originalApplicationNumber',
 'originalApplicationDate',
 'classCode',
 'titleOfInvention',
 'titleOfInventionEng',
 'claimCount',
 'priorityCountry',
 'priorityDate',
 'priorityCount']

In [77]:
right_idc_info_=list(content['registrationRightInfo'].values())

In [78]:
right_idc_info_

['1016988410000',
 None,
 '20170117',
 '20161125',
 '20350519',
 None,
 None,
 '1020150069853',
 None,
 '20150519',
 None,
 '20170123',
 None,
 None,
 None,
 None,
 'D06F 58/26',
 '건조 장치',
 'Drying device',
 '11',
 None,
 None,
 '0']

In [79]:
content['registrationLastRightHolderInfo']

{'lastRightHolderNumber': '120040186134',
 'lastRightHolderName': '주식회사 코나솔',
 'lastRightHolderAddress': '충청남도 당진...',
 'lastRightHolderCountry': None}

In [9]:
request_get

{'response': {'header': {'resultCode': '22',
   'resultMsg': '무료로 사용가능한 요청횟수를 초과하였습니다(Limited number of service request exceeds error)'},
  'body': {'items': {'registrationInfo': {'registrationRightHolderInfo': None}}}}}

In [8]:
content

{'items': {'registrationInfo': {'registrationRightHolderInfo': None}}}

In [4]:
def item_check(url):
    request_get=requests.get(url)
    content_xml=request_get.content
    body=list(xmltodict.parse(content_xml)['response']['body'])[0]
    content=xmltodict.parse(content_xml)['response']['body'][f'{body}']
    
    if content is None:
        print("data is None")
    else:
        detail_name=list(content)[0]
        content=content[detail_name]
        return content

In [5]:
test=item_check(url)

In [6]:
test.keys()

dict_keys(['registrationRightHolderInfo'])

In [7]:
test

{'registrationRightHolderInfo': None}

In [7]:
detail_name

'items'

In [8]:
content=request_get['response']['body']

In [9]:
di_content=content['items']['registrationInfo'].keys()

In [10]:
aa=content['items']['registrationInfo']

In [11]:
aa['registrationRightHolderInfo']['registrationRightHolderInfoB']

TypeError: 'NoneType' object is not subscriptable

In [32]:
di_content

dict_keys(['registrationRightInfo', 'registrationRightHolderInfo', 'registrationRightRankInfo', 'registrationFeeInfo', 'registrationLastRightHolderInfo'])

In [57]:
aa['registrationFeeInfo']

[{'registrationDate': '19920409',
  'startAnnual': '1',
  'lastAnnual': '3',
  'paymentDegree': '1',
  'paymentFee': '31500',
  'paymentDate': '19920409'},
 {'registrationDate': '19950112',
  'startAnnual': '4',
  'lastAnnual': '4',
  'paymentDegree': '1',
  'paymentFee': '28000',
  'paymentDate': '19950112'},
 {'registrationDate': '19960115',
  'startAnnual': '5',
  'lastAnnual': '5',
  'paymentDegree': '1',
  'paymentFee': '28000',
  'paymentDate': '19960115'},
 {'registrationDate': '19961220',
  'startAnnual': '6',
  'lastAnnual': '6',
  'paymentDegree': '1',
  'paymentFee': '28000',
  'paymentDate': '19961220'},
 {'registrationDate': '19980115',
  'startAnnual': '7',
  'lastAnnual': '7',
  'paymentDegree': '1',
  'paymentFee': '93000',
  'paymentDate': '19980115'},
 {'registrationDate': '19990715',
  'startAnnual': '8',
  'lastAnnual': '8',
  'paymentDegree': '1',
  'paymentFee': '186000',
  'paymentDate': '19990716'},
 {'registrationDate': '20000107',
  'startAnnual': '9',
  'last

In [15]:
content['registrationInfo']['registrationRightInfo']

{'registrationNumber': '2000642310000',
 'regReferenceNumber': None,
 'registrationDate': '19920409',
 'assessmentDate': '19920406',
 'expirationDate': '20040506',
 'terminationCauseName': '등록료불납',
 'terminationDate': '20030116',
 'applicationNumber': '2019890005971',
 'appReferenceNumber': None,
 'applicationDate': '19890506',
 'publicationNumber': '2019920000384',
 'publicationDate': '19920115',
 'internationRegistrationNumber': None,
 'internationRegistrationDate': None,
 'originalApplicationNumber': None,
 'originalApplicationDate': None,
 'classCode': 'F04D 29/66',
 'titleOfInvention': '송풍기의소음저감구조',
 'titleOfInventionEng': None,
 'claimCount': '1',
 'priorityCountry': None,
 'priorityDate': None,
 'priorityCount': '0'}

In [51]:
aa['registrationLastRightHolderInfo']

{'lastRightHolderNumber': '119980032017',
 'lastRightHolderName': '주식회사 엑타',
 'lastRightHolderAddress': '서울특별시 강남구...',
 'lastRightHolderCountry': '대한민국'}